In [ ]:
!pip install -U git+https://github.com/huggingface/diffusers git+https://github.com/Disty0/sdnq

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-ty1b4pn8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-ty1b4pn8
  Resolved https://github.com/huggingface/diffusers to commit be3c2a0667493022f17d756ca3dba631d28dfb40
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/Disty0/sdnq to /tmp/pip-req-build-77y5qq2r
  Running command git clone --filter=blob:none --quiet https://github.com/Disty0/sdnq /tmp/pip-req-build-77y5qq2r
  Resolved https://github.com/Disty0/sdnq to commit 6a5be4441c3a25138214eca58270da61c88b65d4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-py3-none-any.whl size=4636030 sha256=1ff70791a7e08b74e1563954d65280a3e3cff13cb50

In [ ]:
!pip install flask flask-cors diffusers transformers accelerate sdnq pyngrok --quiet

In [ ]:
from pyngrok import ngrok

# Input Grok Token
ngrok.set_auth_token("TOKEN NGROK")


In [ ]:
import torch
import diffusers
from sdnq import SDNQConfig
from sdnq.loader import apply_sdnq_options_to_model

# --- LOAD MODEL One Time ---
def load_pipeline():
    pipe = diffusers.ZImagePipeline.from_pretrained(
        "Disty0/Z-Image-Turbo-SDNQ-uint4-svd-r32",
        torch_dtype=torch.float32,
        device_map="cuda"
    )

    pipe.transformer = apply_sdnq_options_to_model(pipe.transformer, use_quantized_matmul=True)
    pipe.text_encoder = apply_sdnq_options_to_model(pipe.text_encoder, use_quantized_matmul=True)

    return pipe

pipe = load_pipeline()
print("Pipeline ready to use!")


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

scheduler_config.json:   0%|          | 0.00/487 [00:00<?, ?B/s]

...

transformer/diffusion_pytorch_model.safe(…):   0%|          | 0.00/3.48G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Pipeline siap digunakan!


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
from io import BytesIO

app = Flask(__name__)
CORS(app)

@app.route('/generate', methods=['POST'])
def generate_image():
    data = request.json

    prompt = data.get("prompt", "default prompt")
    height = data.get("height", 1024)
    width = data.get("width", 1024)

    try:
        image = pipe(
            prompt=prompt,
            height=height,
            width=width,
            num_inference_steps=9,
            guidance_scale=0.0,
            generator=torch.manual_seed(42),
        ).images[0]

        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

        return jsonify({
            "status": "success",
            "prompt": prompt,
            "image_base64": img_base64
        })

    except Exception as e:
        return jsonify({"status": "error", "message": str(e)})


In [ ]:
# Jalankan Flask
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app.run(port=5000)


For simple use

> curl -X POST https://xxxxx.ngrok-free.app/generate \
-H "Content-Type: application/json" \
-d '{"prompt": "kodak"}'

or advance use

> curl -X POST "https://f42b1e124a6f.ngrok-free.app/generate" \
  -H "Content-Type: application/json" \
  -d '{
  "prompt": "A warm and cozy wooden Christmas café at night, viewed from a single fixed camera angle. The camera does not move. A beautifully decorated Christmas tree with colorful lights glows in the center. A fireplace with a gentle flame adds warm light. Candles on tables, wrapped gifts, plaid blankets, and festive decorations are visible. Snow is falling softly outside the open wooden structure. Warm yellow lighting, wooden textures, Christmas garlands and wreaths. Ultra-realistic, 4K, peaceful ambience. No camera movement, only natural ambient motions like flickering fire, candlelight, and falling snow.",
  "width": 1024,
  "height": 576
}'

